In [ ]:
# Setup imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='whitegrid')
DATA_DIR = Path('../data/raw')
print('Notebook running from:', Path.cwd())

Notebook running from: /Users/bofeng/Downloads/junction_fortum/notebooks


In [9]:
# Find Excel files in data/raw
excel_files = list(DATA_DIR.glob('*.xls*'))
print(f'Found {len(excel_files)} Excel file(s) in {DATA_DIR}')
for p in excel_files:
    print('-', p.name)

if not excel_files:
    raise FileNotFoundError(f'No Excel files found in {DATA_DIR}; please add your training file to data/raw')

Found 1 Excel file(s) in ../data/raw
- 20251111_JUNCTION_training.xlsx


In [15]:
df.head()

,group_id,group_label
0,28,Eastern Finland | Etelä-Savo | Etelä-Savo | Pr...
1,29,Eastern Finland | Etelä-Savo | Etelä-Savo | Pr...
2,30,Eastern Finland | Etelä-Savo | Etelä-Savo | Pr...
3,36,Eastern Finland | Pohjois-Karjala | Joensuu | ...
4,37,Eastern Finland | Pohjois-Karjala | Joensuu | ...


In [11]:
# Load the first Excel file (with fallback to install openpyxl)
file_path = excel_files[0]
try:
    df = pd.read_excel(file_path)
except ValueError as e:
    # sometimes engine needs openpyxl for .xlsx
    print('Read error, attempting to ensure openpyxl is installed:', e)
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'openpyxl'])
    df = pd.read_excel(file_path)

print('Loaded:', file_path.name)
print('Rows, cols:', df.shape)

Loaded: 20251111_JUNCTION_training.xlsx
Rows, cols: (112, 2)


In [12]:
# Quick peek
display(df.head())

# Basic info
print('
-- info --')
display(df.info())

print('
-- describe (numeric) --')
display(df.describe(include=[np.number]).T)

SyntaxError: unterminated string literal (detected at line 5) (470733708.py, line 5)

In [13]:
# Missing values summary
na_counts = df.isna().sum().sort_values(ascending=False)
display(na_counts[na_counts>0])

# Column datatypes
display(df.dtypes)

Series([], dtype: int64)

group_id        int64
group_label    object
dtype: object

In [14]:
# Try to detect a datetime column and set as index if present
time_cols = [c for c in df.columns if c.lower() in ('timestamp', 'datetime', 'date', 'time')]
if not time_cols:
    # heuristic: object columns that parse to datetime
    for c in df.select_dtypes(include=['object']).columns:
        try:
            pd.to_datetime(df[c].dropna().iloc[:50])
            time_cols.append(c)
        except Exception:
            pass

print('Detected time columns:', time_cols)

if time_cols:
    tcol = time_cols[0]
    df[tcol] = pd.to_datetime(df[tcol], errors='coerce')
    df = df.sort_values(tcol).set_index(tcol)
    display(df.head())
    # simple time series plot for numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()[:3]
    if numeric_cols:
        df[numeric_cols].plot(subplots=False, figsize=(12,4));
        plt.title('First numeric columns over time')
else:
    print('No time column detected; skipping time-series plots')

Detected time columns: []
No time column detected; skipping time-series plots


/var/folders/9m/2x_kqxt17hvg2pn0gfbz1ml80000gn/T/ipykernel_75013/2552799472.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[c].dropna().iloc[:50])


In [ ]:
# Target/feature check
possible_target = 'energy_consumption'
if possible_target in df.columns:
    sns.histplot(df[possible_target].dropna(), kde=True);
    plt.title(possible_target)
else:
    print('Target column `energy_consumption` not found. Available columns:')
    print(list(df.columns)[:30])

## Next steps
- Clean and process missing values (`data/processed`).
- Use `src/features/build_features.py` for deterministic transforms.
- Run `src/models/train.py` with `configs/default.yaml` after preparing the processed dataset.